In [1]:
# import pdfplumber
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords 
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict
import pandas as pd 
from nltk.collocations import *
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [2]:
jd='''Role - Senior Big Data Engineer
Required Technical skills - Big Data, Python, Spark, Data brick, data Spark
Desired Experience Range - 5 + years
Location of Requirement - PAN India
Desired Competencies
Must-Have
Good hands on experience in Python programming
· Data Engineering experience using AWS core services (Lambda, Glue, EMR and RedShift)
· Required skill set- SQL, Airflow · Must – Have Experience with snowflake

· Responsibility – Will be accountable for build/test complex data pipelines (batch and near real time) · Expectation – Readable documentation of all the components being develop · Experience in writing SQLs and stored procedures · Working experience with RDBMS (Oracle / Teradata)
Good-to-Have
· Good understanding of Data warehouse
· Familiarize in ETL tools like Informatica
· Experience working with NoSQL database like DynamoDB or MongoDB .'''

In [3]:


def remove_stuff(jd):
    jd_clean = jd.replace("\xa0", "").replace("/", "").replace(".", ". ").replace("●", "")
    return jd_clean

In [4]:
def get_cv(): #cleaned, processed, nlped cv content
    url='updated_cv.csv'
    return pd.read_csv(url) 
# db_expander = st.beta_expander(label='Submitted resume:')
# with db_expander:
df = get_cv()

# checking data

In [5]:
# cv data

In [6]:
df

,Unnamed: 0,name,email,mobile_number,skills,education,experience_years,file_path,raw_text,pdf_to_base64
0,0,Le Thanh Long,attentionocr@gmail.com,096-997-3012,"Mock, Research, Communication, Scheduling, Css...",[],0.00,cv\1.0.pdf,Le Thanh Long Research and Development Phone09...,JVBERi0xLjQKJaGzxdcKMSAwIG9iag0KPDwvVHlwZS9DYX...
1,1,Le Thanh Long,attentionocr@gmail.com,096-997-3012,"Research, Communication, Scheduling, Css, Flas...",['MLOPS'],0.00,cv\2.0.pdf,Le Thanh Long Research and Development Birth26...,JVBERi0xLjQKJcLB2s/OCgoxIDAgb2JqIDw8CiAgL1R5cG...
2,2,NHAN HO,hosybaonhan2004@gmail.com,022-2027,"Linux, Research, Communication, Flask, Mobile,...",[],0.00,cv\bao nhan ho sy-cv.pdf,BAO-NHAN HO-SY PYTHON BACK-END DEVELOPER Short...,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...
3,3,thanhduycao |,ctduy20@apcs.fitus.edu.vn,NaN,"Research, Content, Communication, Flask, Syste...",[],1.08,cv\CaoThanhDuy_Resume.pdf,THANH-DUY CAO ctduy20@apcs.fitus.edu.vn thanh...,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...
4,4,NGUYEN THI,chauthi1704@gmail.com,8438728569,"Communication, Css, User experience, Mobile, S...",[],0.00,cv\ChauThi_resume.pdf,"NGUYEN THI CHAU THI 470, Tran Dai Nghia, Ngu H...",JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...
5,5,Lê Thùy Dương,ltduong2911@gmail.com,0707292696,"Website, Lan, Email, Word, Excel",[],0.00,cv\CV Thuy Duong (Việt)1743071666667.pdf,Lê Thùy Dương Nhân viên văn phòng Ngày sinh 29...,CiVQREYtMS43CiWBgYGBCgo0IDAgb2JqCjw8Ci9UeXBlIC...
6,6,Dao Khanh Duy,khanhhh.duy@gmail.com,022-2026,"Docker, Communication, Css, User experience, E...",['Bachelor of Science in Computer Science'],1.42,cv\DaoKhanhDuy.pdf,Dao Khanh Duy Sept 2004 0789-402-189 khanhhh...,JVBERi0xLjUKJb/3ov4KNSAwIG9iago8PCAvTGluZWFyaX...
7,7,Ho Quoc Thien Anh,hoquocthienanh@gmail.com,0905500293,"Hotel, Communication, Seaborn, English, Presen...","[""Bachelor's degree Information Technology""]",2.58,cv\HoQuocThienAnh.pdf,"Ho Quoc Thien Anh Vietnam, Da Nang City hoquo...",JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/KQovQ3...
8,8,HUYNH THI,huynhthiphuoc04@gmail.com,022-2026,"Teaching, Java, Workflow, Docker, Php, Css, Co...",[],1.25,cv\huynhthiphuoc_resume1.pdf,HUYNH THI PHUOC July 2004 0355-176-782 huynh...,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...
9,9,Nguyen Tien,dungbk.nguyentien@g,0973621049,"Linux, Research, Docker, Unix, Agile, System, ...",['\uf0b1'],0.00,cv\Nguyen-Tien-Dung.pdf,Nguyen Tien Dung 0973621049 Male Developer ...,JVBERi0xLjcKJYGBgYEKCjQgMCBvYmoKPDwKL1R5cGUgL1...


In [7]:
df["All"]

KeyError: 'All'

In [ ]:
# Xem 5 dòng đầu
print(df.head())

# Kiểm tra tên cột
print("Columns:", df.columns.tolist())

# Kiểm tra giá trị null
print("Missing values:", df.isnull().sum())

   Unnamed: 0           name                      email mobile_number  \
0           0       Le Thanh     attentionocr@gmail.com  096-997-3012   
1           1       Le Thanh     attentionocr@gmail.com  096-997-3012   
2           2        NHAN HO  hosybaonhan2004@gmail.com      022-2027   
3           3  thanhduycao |  ctduy20@apcs.fitus.edu.vn           NaN   
4           4     NGUYEN THI      chauthi1704@gmail.com    8438728569   

                                              skills     degree  no_of_pages  \
0  ['Product owner', 'Ai', 'Tensorflow', 'Chemist...       None            5   
1  ['Ai', 'Certification', 'Tensorflow', 'Chemist...  ['MLOPS']            6   
2  ['English', 'Website', 'Scrum', 'Postgresql', ...       None            2   
3  ['Apis', 'Video', 'Ai', 'Statistics', 'Experim...       None            2   
4  ['Apis', 'Access', 'English', 'Ui', 'Database'...       None            2   

                   file_path  \
0                 cv\1.0.pdf   
1               

In [ ]:
df["All"] = df["All"].str.replace("\r\n", " ")  # Thay thế xuống dòng
df["All"] = df["All"].str.replace("â¢", "-")   # Sửa lỗi encoding
df["All"] = df["All"].str.lower()
import string
df["All"] = df["All"].str.translate(str.maketrans("", "", string.punctuation))

In [ ]:
# job discription entered by recuriter

In [ ]:
jd

'Role - Senior Big Data Engineer\nRequired Technical skills - Big Data, Python, Spark, Data brick, data Spark\nDesired Experience Range - 5 + years\nLocation of Requirement - PAN India\nDesired Competencies\nMust-Have\nGood hands on experience in Python programming\n· Data Engineering experience using AWS core services (Lambda, Glue, EMR and RedShift)\n· Required skill set- SQL, Airflow · Must – Have Experience with snowflake\n\n· Responsibility – Will be accountable for build/test complex data pipelines (batch and near real time) · Expectation – Readable documentation of all the components being develop · Experience in writing SQLs and stored procedures · Working experience with RDBMS (Oracle / Teradata)\nGood-to-Have\n· Good understanding of Data warehouse\n· Familiarize in ETL tools like Informatica\n· Experience working with NoSQL database like DynamoDB or MongoDB .'

In [ ]:
def nlp(x):
    word_sent = word_tokenize(x.lower().replace("\n",""))
    _stopwords = set(stopwords.words('english') + list(punctuation)+list("●")+list('–')+list('’'))
    word_sent=[word for word in word_sent if word not in _stopwords]
    lemmatizer = WordNetLemmatizer()
    clean_text = [lemmatizer.lemmatize(word) for word in word_tokenize(" ".join(word_sent))]
#     return " ".join(NLP_Processed_CV)
    return clean_text

In [ ]:
NLP_Processed_JD=nlp(jd)


In [ ]:
NLP_Processed_JD

['role',
 'senior',
 'big',
 'data',
 'engineerrequired',
 'technical',
 'skill',
 'big',
 'data',
 'python',
 'spark',
 'data',
 'brick',
 'data',
 'sparkdesired',
 'experience',
 'range',
 '5',
 'yearslocation',
 'requirement',
 'pan',
 'indiadesired',
 'competenciesmust-havegood',
 'hand',
 'experience',
 'python',
 'programming·',
 'data',
 'engineering',
 'experience',
 'using',
 'aws',
 'core',
 'service',
 'lambda',
 'glue',
 'emr',
 'redshift',
 '·',
 'required',
 'skill',
 'set-',
 'sql',
 'airflow',
 '·',
 'must',
 'experience',
 'snowflake·',
 'responsibility',
 'accountable',
 'build/test',
 'complex',
 'data',
 'pipeline',
 'batch',
 'near',
 'real',
 'time',
 '·',
 'expectation',
 'readable',
 'documentation',
 'component',
 'develop',
 '·',
 'experience',
 'writing',
 'sqls',
 'stored',
 'procedure',
 '·',
 'working',
 'experience',
 'rdbms',
 'oracle',
 'teradata',
 'good-to-have·',
 'good',
 'understanding',
 'data',
 'warehouse·',
 'familiarize',
 'etl',
 'tool',
 'li

In [ ]:
################################################################################################################################

In [ ]:
jd_df=pd.DataFrame()
# jd_df['hi']=['I']
jd_df['jd']=[' '.join(NLP_Processed_JD)]

In [ ]:
jd_df['jd'].values[0]

'role senior big data engineerrequired technical skill big data python spark data brick data sparkdesired experience range 5 yearslocation requirement pan indiadesired competenciesmust-havegood hand experience python programming· data engineering experience using aws core service lambda glue emr redshift · required skill set- sql airflow · must experience snowflake· responsibility accountable build/test complex data pipeline batch near real time · expectation readable documentation component develop · experience writing sqls stored procedure · working experience rdbms oracle teradata good-to-have· good understanding data warehouse· familiarize etl tool like informatica· experience working nosql database like dynamodb mongodb'

In [ ]:
clean_jd=jd_df['jd'].values[0]

In [ ]:
##############################################################################################################################

In [ ]:
df["All"]

0     le thanh longresearch and developmentphone 096...
1     le thanh longresearch and developmentbirth 260...
2     baonhan hosypython backend developershortterm ...
3     thanhduy caoctduy20apcsfituseduvn  thanhduycao...
4     nguyen thi chau thi470 tran dai nghia ngu hanh...
5     l thy dngnhn vin vn phngngy sinh 29111999gii t...
6     ho quoc thien anhvietnam da nang city  hoquoct...
7     huynh thi phuocjuly 2004  0355176782  huynhthi...
8     nguyen tien dung  0973621049  maledeveloper du...
9     nguyen van nghiemai engineer 0399802905  nguye...
10    nguyen hoang phuc cid211 84975594053computer s...
11    phung duc hai84 948601960  haiphung15012002gma...
12    pham quang tuinterndob 26092002gender malephon...
13    p han minh nht  02092003 nhat0299gmailcom da n...
14    objectivesto gain handson experience in fronte...
15    phuoc phanangular developer  3 years of experi...
16    quang phong nguyenapartment a5 vicoland van do...
17    le thanh long email  attentionocrgmailcoml

In [ ]:
cv_data=[]
for i in range(len(df["All"])):
    NLP_Processed_cv=nlp(df["All"].values[i])
    cv_data.append(NLP_Processed_cv)

In [ ]:
cv_data

[['le',
  'thanh',
  'longresearch',
  'developmentphone',
  '0969973012',
  'date',
  'birth',
  '26092001address',
  'ha',
  'noi',
  'viet',
  'namemail',
  'attentionocrgmailcomoverviewexperience',
  'programming',
  'good',
  'communication',
  'quick',
  'learning',
  'skillshave',
  'quality',
  'team',
  'leadersharp',
  'thinking',
  'many',
  'outstanding',
  'creative',
  'ideashigh',
  'spirit',
  'learningjob',
  'goalsshortterm',
  'stable',
  'job',
  'become',
  'excellent',
  'employee',
  'accumulate',
  'moreexperiencelongterm',
  'become',
  'manager',
  'join',
  'management',
  'executive',
  'team',
  'thecompanyeducation2019',
  'present',
  'hanoi',
  'university',
  'science',
  'technologymajoring',
  'computer',
  'sciencecurrently',
  'cpa',
  '362skillsproblem',
  'analysis',
  'ability',
  'analyze',
  'problem',
  'quickly',
  'effectivelyproject',
  'scheduling',
  'almost',
  'project',
  'led',
  'finished',
  'due',
  'timeprogramming',
  'webapp',
 

In [ ]:
cv_=[]
for i in cv_data:
    cv_.append([' '.join(i)])

In [ ]:
cv_

[['le thanh longresearch developmentphone 0969973012 date birth 26092001address ha noi viet namemail attentionocrgmailcomoverviewexperience programming good communication quick learning skillshave quality team leadersharp thinking many outstanding creative ideashigh spirit learningjob goalsshortterm stable job become excellent employee accumulate moreexperiencelongterm become manager join management executive team thecompanyeducation2019 present hanoi university science technologymajoring computer sciencecurrently cpa 362skillsproblem analysis ability analyze problem quickly effectivelyproject scheduling almost project led finished due timeprogramming webapp artificial intelligencereporting maker strong proficiency making project reportpresentation good presentation skill bring product idea people effectivelycertifications052023 ai platform kaggle competion expert042023 052023 top 2 track cyber security cyberphish shield junctionxhanoi hackathon 2023012023 022023 1nd prize nghe hackath

In [ ]:
df["clean_all"]=pd.DataFrame(cv_)

# Recomendation Function

In [ ]:
df.head(1)

,Unnamed: 0,name,email,mobile_number,skills,degree,no_of_pages,file_path,All,pdf_to_base64,clean_all
0,0,Le Thanh,attentionocr@gmail.com,096-997-3012,"['Product owner', 'Ai', 'Tensorflow', 'Chemist...",None,5,cv\1.0.pdf,le thanh longresearch and developmentphone 096...,JVBERi0xLjQKJaGzxdcKMSAwIG9iag0KPDwvVHlwZS9DYX...,le thanh longresearch developmentphone 0969973...


In [ ]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['name', 'degree',"email",'Unnamed: 0','mobile_number','skills','no_of_pages','score'])
    count = 0
    for i in top:
#         recommendation.at[count, 'ApplicantID'] = u
        
        recommendation.at[count, 'name'] = df['name'][i]
        recommendation.at[count, 'degree'] = df['degree'][i]
        recommendation.at[count, 'email'] = df['email'][i]
        recommendation.at[count, 'Unnamed: 0'] = df.index[i]
        recommendation.at[count, 'mobile_number'] = df['mobile_number'][i]
        recommendation.at[count, 'skills'] = df['skills'][i]
        recommendation.at[count, 'no_of_pages'] = df['no_of_pages'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

# cosine_similarity + TfidfVectorizer function

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


def TFIDF(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # TF-IDF Scraped data
    tfidf_jobid = tfidf_vectorizer.fit_transform(scraped_data)

    # TF-IDF CV
    user_tfidf = tfidf_vectorizer.transform(cv)

    # Using cosine_similarity on (Scraped data) & (CV)
    cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf,x),tfidf_jobid)

    output2 = list(cos_similarity_tfidf)
    return output2
    


In [ ]:
output2 = TFIDF(df["clean_all"],jd_df['jd'])

In [ ]:
output2 

[array([[0.12721305]]),
 array([[0.12063487]]),
 array([[0.05914294]]),
 array([[0.06883416]]),
 array([[0.0662449]]),
 array([[0.]]),
 array([[0.11534099]]),
 array([[0.07520907]]),
 array([[0.09709831]]),
 array([[0.0751482]]),
 array([[0.08630985]]),
 array([[0.13340926]]),
 array([[0.02710227]]),
 array([[0.06139357]]),
 array([[0.03035652]]),
 array([[0.08456258]]),
 array([[0.02756394]]),
 array([[0.19293692]]),
 array([[0.08731025]])]

In [ ]:
print("👉 Các cột trong df:", df.columns.tolist())

👉 Các cột trong df: ['Unnamed: 0', 'name', 'email', 'mobile_number', 'skills', 'degree', 'no_of_pages', 'file_path', 'All', 'pdf_to_base64', 'clean_all']


In [ ]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:100]
list_scores = [output2[i][0][0] for i in top]
TF=get_recommendation(top,df, list_scores)
TF

,name,degree,email,Unnamed: 0,mobile_number,skills,no_of_pages,score
0,Le Thanh,['Bachelor of Computer Science; CPA: 3.65nnExp...,attentionocr@gmail.com,17,9699730,"['Ai', 'Certification', 'Try', 'Computer scien...",2,0.192937
1,| linkedin.com/in/haiph02,['Bachelor of Science in Information Technolog...,haiphung15012002@gmail.com,11,9486019,"['English', 'Queries', 'Scheduling', 'Saas', '...",1,0.133409
2,Le Thanh,None,attentionocr@gmail.com,0,096-997-3012,"['Product owner', 'Ai', 'Tensorflow', 'Chemist...",5,0.127213
3,Le Thanh,['MLOPS'],attentionocr@gmail.com,1,096-997-3012,"['Ai', 'Certification', 'Tensorflow', 'Chemist...",6,0.120635
4,Deputy Secretary,"[""Bachelor's degree Information Technology""]",hoquocthienanh@gmail.com,6,0905500293,"['Pandas', 'Ai', 'Forecasting', 'Seaborn', 'En...",1,0.115341
5,Nguyen Tien,['uf0b1'],dungbk.nguyentien@g,8,0973621049,"['Video', 'Testing', 'C', 'Servers', 'Try', 'J...",2,0.097098
6,Đà Nang,['Bachelor of Information Technology Engineeri...,tranghuyen9696@gmail.com,18,222 5469,"['English', 'Website', 'Java', 'Analysis', 'Pr...",1,0.08731
7,Nguyen Hoang,['Computer Science'],phuc.nh215452@sis.hust.edu.vn,10,9755940,"['Health', 'Ai', 'Tensorflow', 'C', 'Pycharm',...",1,0.08631
8,Phuoc Phan,['2022'],"zVGEJ@EJS,",15,NaN,"['English', 'Customer management', 'R', 'Logis...",3,0.084563
9,HUYNH THI,None,huynhthiphuoc04@gmail.com,7,022-2026,"['Css', 'Html', 'Health', 'Java', 'Project man...",2,0.075209


In [ ]:
df

,Unnamed: 0,name,email,mobile_number,skills,degree,no_of_pages,file_path,All,pdf_to_base64,clean_all
0,0,Le Thanh,attentionocr@gmail.com,096-997-3012,"['Product owner', 'Ai', 'Tensorflow', 'Chemist...",None,5,cv\1.0.pdf,le thanh longresearch and developmentphone 096...,JVBERi0xLjQKJaGzxdcKMSAwIG9iag0KPDwvVHlwZS9DYX...,le thanh longresearch developmentphone 0969973...
1,1,Le Thanh,attentionocr@gmail.com,096-997-3012,"['Ai', 'Certification', 'Tensorflow', 'Chemist...",['MLOPS'],6,cv\2.0.pdf,le thanh longresearch and developmentbirth 260...,JVBERi0xLjQKJcLB2s/OCgoxIDAgb2JqIDw8CiAgL1R5cG...,le thanh longresearch developmentbirth 2609200...
2,2,NHAN HO,hosybaonhan2004@gmail.com,022-2027,"['English', 'Website', 'Scrum', 'Postgresql', ...",None,2,cv\bao nhan ho sy-cv.pdf,baonhan hosypython backend developershortterm ...,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...,baonhan hosypython backend developershortterm ...
3,3,thanhduycao |,ctduy20@apcs.fitus.edu.vn,NaN,"['Apis', 'Video', 'Ai', 'Statistics', 'Experim...",None,2,cv\CaoThanhDuy_Resume.pdf,thanhduy caoctduy20apcsfituseduvn thanhduycao...,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...,thanhduy caoctduy20apcsfituseduvn thanhduycao ...
4,4,NGUYEN THI,chauthi1704@gmail.com,8438728569,"['Apis', 'Access', 'English', 'Ui', 'Database'...",None,2,cv\ChauThi_resume.pdf,nguyen thi chau thi470 tran dai nghia ngu hanh...,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...,nguyen thi chau thi470 tran dai nghia ngu hanh...
5,5,Lê Thùy,ltduong2911@gmail.com,0707292696,"['Excel', 'Email', 'Word', 'Lan', 'Website']",None,2,cv\CV Thuy Duong (Việt)1743071666667.pdf,l thy dngnhn vin vn phngngy sinh 29111999gii t...,CiVQREYtMS43CiWBgYGBCgo0IDAgb2JqCjw8Ci9UeXBlIC...,l thy dngnhn vin vn phngngy sinh 29111999gii t...
6,6,Deputy Secretary,hoquocthienanh@gmail.com,0905500293,"['Pandas', 'Ai', 'Forecasting', 'Seaborn', 'En...","[""Bachelor's degree Information Technology""]",1,cv\HoQuocThienAnh.pdf,ho quoc thien anhvietnam da nang city hoquoct...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/KQovQ3...,ho quoc thien anhvietnam da nang city hoquocth...
7,7,HUYNH THI,huynhthiphuoc04@gmail.com,022-2026,"['Css', 'Html', 'Health', 'Java', 'Project man...",None,2,cv\huynhthiphuoc_resume1.pdf,huynh thi phuocjuly 2004 0355176782 huynhthi...,JVBERi0xLjcNCiW1tbW1DQoxIDAgb2JqDQo8PC9UeXBlL0...,huynh thi phuocjuly 2004 0355176782 huynhthiph...
8,8,Nguyen Tien,dungbk.nguyentien@g,0973621049,"['Video', 'Testing', 'C', 'Servers', 'Try', 'J...",['uf0b1'],2,cv\Nguyen-Tien-Dung.pdf,nguyen tien dung 0973621049 maledeveloper du...,JVBERi0xLjcKJYGBgYEKCjQgMCBvYmoKPDwKL1R5cGUgL1...,nguyen tien dung 0973621049 maledeveloper dung...
9,9,NGUYEN VAN,Engineer0399802905✉nguyenvannghiem0312@gmail....,0399802905,"['Testing', 'Ai', 'Tensorflow', 'C', 'Recruitm...",None,2,cv\NguyenVanNghiem.pdf,nguyen van nghiemai engineer 0399802905 nguye...,JVBERi0xLjcKJb/3ov4KMSAwIG9iago8PCAvVHlwZSAvT2...,nguyen van nghiemai engineer 0399802905 nguyen...


# cosine_similarity + CountVectorizer


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer()
count_jobid = count_vectorizer.fit_transform(df["clean_all"]) #converting job data into vectors using count vectorizers
count_jobid

<19x2758 sparse matrix of type '<class 'numpy.int64'>'
	with 4398 stored elements in Compressed Sparse Row format>

In [ ]:
user_count = count_vectorizer.transform(jd_df['jd'])#converting user cv data into vectors using count vectorizers
user_count

<1x2758 sparse matrix of type '<class 'numpy.int64'>'
	with 36 stored elements in Compressed Sparse Row format>

In [ ]:
print(user_count)

  (0, 292)	1
  (0, 462)	1
  (0, 509)	1
  (0, 667)	1
  (0, 669)	1
  (0, 706)	1
  (0, 762)	7
  (0, 763)	1
  (0, 830)	1
  (0, 879)	1
  (0, 972)	1
  (0, 1028)	7
  (0, 1183)	2
  (0, 1522)	2
  (0, 1658)	1
  (0, 1762)	1
  (0, 1889)	1
  (0, 1980)	1
  (0, 2016)	2
  (0, 2056)	1
  (0, 2110)	1
  (0, 2123)	1
  (0, 2145)	1
  (0, 2210)	1
  (0, 2221)	1
  (0, 2248)	2
  (0, 2308)	1
  (0, 2331)	1
  (0, 2412)	1
  (0, 2437)	1
  (0, 2490)	1
  (0, 2513)	1
  (0, 2587)	1
  (0, 2613)	1
  (0, 2730)	2
  (0, 2737)	1


In [ ]:
cos_similarity_countv = map(lambda x: cosine_similarity(user_count, x),count_jobid)
output3 = list(cos_similarity_countv)
#output3

In [ ]:
output3 

[array([[0.20410007]]),
 array([[0.19589216]]),
 array([[0.09875429]]),
 array([[0.11352458]]),
 array([[0.10522707]]),
 array([[0.]]),
 array([[0.23868686]]),
 array([[0.08739647]]),
 array([[0.17600812]]),
 array([[0.1460104]]),
 array([[0.1694515]]),
 array([[0.23809524]]),
 array([[0.07318121]]),
 array([[0.12392754]]),
 array([[0.04486245]]),
 array([[0.14520228]]),
 array([[0.06182033]]),
 array([[0.31939755]]),
 array([[0.14197833]])]

In [ ]:
top = sorted(range(len(output3)), key=lambda i: output3[i], reverse=True)[:100]
list_scores = [output3[i][0][0] for i in top]
cv=get_recommendation(top, df, list_scores)
cv

,name,degree,email,Unnamed: 0,mobile_number,skills,no_of_pages,score
0,Le Thanh,['Bachelor of Computer Science; CPA: 3.65nnExp...,attentionocr@gmail.com,17,9699730,"['Ai', 'Certification', 'Try', 'Computer scien...",2,0.319398
1,Deputy Secretary,"[""Bachelor's degree Information Technology""]",hoquocthienanh@gmail.com,6,0905500293,"['Pandas', 'Ai', 'Forecasting', 'Seaborn', 'En...",1,0.238687
2,| linkedin.com/in/haiph02,['Bachelor of Science in Information Technolog...,haiphung15012002@gmail.com,11,9486019,"['English', 'Queries', 'Scheduling', 'Saas', '...",1,0.238095
3,Le Thanh,None,attentionocr@gmail.com,0,096-997-3012,"['Product owner', 'Ai', 'Tensorflow', 'Chemist...",5,0.2041
4,Le Thanh,['MLOPS'],attentionocr@gmail.com,1,096-997-3012,"['Ai', 'Certification', 'Tensorflow', 'Chemist...",6,0.195892
5,Nguyen Tien,['uf0b1'],dungbk.nguyentien@g,8,0973621049,"['Video', 'Testing', 'C', 'Servers', 'Try', 'J...",2,0.176008
6,Nguyen Hoang,['Computer Science'],phuc.nh215452@sis.hust.edu.vn,10,9755940,"['Health', 'Ai', 'Tensorflow', 'C', 'Pycharm',...",1,0.169452
7,NGUYEN VAN,None,Engineer0399802905✉nguyenvannghiem0312@gmail....,9,0399802905,"['Testing', 'Ai', 'Tensorflow', 'C', 'Recruitm...",2,0.14601
8,Phuoc Phan,['2022'],"zVGEJ@EJS,",15,NaN,"['English', 'Customer management', 'R', 'Logis...",3,0.145202
9,Đà Nang,['Bachelor of Information Technology Engineeri...,tranghuyen9696@gmail.com,18,222 5469,"['English', 'Website', 'Java', 'Analysis', 'Pr...",1,0.141978


# TfidfVectorizer + NearestNeighbors

In [ ]:
from sklearn.neighbors import NearestNeighbors

def KNN(scraped_data, cv):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    
    KNN = NearestNeighbors(n_neighbors=7,p=2)
    KNN.fit(tfidf_vectorizer.fit_transform(scraped_data))
#     NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv), return_distance=True)
    NNs = KNN.kneighbors(tfidf_vectorizer.transform(cv))
    top = NNs[1][0][1:]
    index_score = NNs[0][0][1:]

    knn = get_recommendation(top, df, index_score)
    return knn

knn = KNN(df['clean_all'], jd_df['jd'])
knn

,name,degree,email,Unnamed: 0,mobile_number,skills,no_of_pages,score
0,| linkedin.com/in/haiph02,['Bachelor of Science in Information Technolog...,haiphung15012002@gmail.com,11,9486019,"['English', 'Queries', 'Scheduling', 'Saas', '...",1,1.316504
1,Le Thanh,None,attentionocr@gmail.com,0,096-997-3012,"['Product owner', 'Ai', 'Tensorflow', 'Chemist...",5,1.321202
2,Le Thanh,['MLOPS'],attentionocr@gmail.com,1,096-997-3012,"['Ai', 'Certification', 'Tensorflow', 'Chemist...",6,1.326171
3,Deputy Secretary,"[""Bachelor's degree Information Technology""]",hoquocthienanh@gmail.com,6,0905500293,"['Pandas', 'Ai', 'Forecasting', 'Seaborn', 'En...",1,1.330157
4,Nguyen Tien,['uf0b1'],dungbk.nguyentien@g,8,0973621049,"['Video', 'Testing', 'C', 'Servers', 'Try', 'J...",2,1.343802
5,Đà Nang,['Bachelor of Information Technology Engineeri...,tranghuyen9696@gmail.com,18,222 5469,"['English', 'Website', 'Java', 'Analysis', 'Pr...",1,1.351066


In [ ]:

knn.to_csv('knn_cv.csv', index=False)

# Combined TFIDF, CV and KNN result together, to make a dataframe "final"

In [ ]:
knn.columns

Index(['name', 'degree', 'email', 'Unnamed: 0', 'mobile_number', 'skills',
       'no_of_pages', 'score'],
      dtype='object')

In [ ]:
merge1 = knn[['Unnamed: 0','name', 'score']].merge(TF[['Unnamed: 0','score']], on= "Unnamed: 0")
final = merge1.merge(cv[['Unnamed: 0','score']], on = 'Unnamed: 0')
final = final.rename(columns={"score_x": "KNN", "score_y": "TF-IDF","score": "CV"})
final.head()

,Unnamed: 0,name,KNN,TF-IDF,CV
0,11,| linkedin.com/in/haiph02,1.316504,0.133409,0.238095
1,0,Le Thanh,1.321202,0.127213,0.2041
2,1,Le Thanh,1.326171,0.120635,0.195892
3,6,Deputy Secretary,1.330157,0.115341,0.238687
4,8,Nguyen Tien,1.343802,0.097098,0.176008


In [ ]:
final

,Unnamed: 0,name,KNN,TF-IDF,CV
0,11,| linkedin.com/in/haiph02,1.316504,0.133409,0.238095
1,0,Le Thanh,1.321202,0.127213,0.2041
2,1,Le Thanh,1.326171,0.120635,0.195892
3,6,Deputy Secretary,1.330157,0.115341,0.238687
4,8,Nguyen Tien,1.343802,0.097098,0.176008
5,18,Đà Nang,1.351066,0.08731,0.141978


In [ ]:
# Scale it
from sklearn.preprocessing import MinMaxScaler
slr = MinMaxScaler()
final[["KNN", "TF-IDF", 'CV']] = slr.fit_transform(final[["KNN", "TF-IDF", 'CV']])

# Multiply by weights
final['KNN'] = (1-final['KNN'])/3
final['TF-IDF'] = final['TF-IDF']/3
final['CV'] = final['CV']/3
final['Final'] = final['KNN']+final['TF-IDF']+final['CV']
final.sort_values(by="Final", ascending=False)

,Unnamed: 0,name,KNN,TF-IDF,CV,Final
0,11,| linkedin.com/in/haiph02,0.333333,0.333333,0.331294,0.997961
1,0,Le Thanh,0.288022,0.288530,0.214120,0.790672
3,6,Deputy Secretary,0.201653,0.202685,0.333333,0.737671
2,1,Le Thanh,0.240094,0.240964,0.185829,0.666887
4,8,Nguyen Tien,0.070059,0.070776,0.117293,0.258128
5,18,Đà Nang,0.000000,0.000000,0.000000,0.000000


In [ ]:
# final.to_csv('final.csv', index=False)
final2 = final.sort_values(by="Final", ascending=False).copy()
final2

,Unnamed: 0,name,KNN,TF-IDF,CV,Final
0,11,| linkedin.com/in/haiph02,0.333333,0.333333,0.331294,0.997961
1,0,Le Thanh,0.288022,0.288530,0.214120,0.790672
3,6,Deputy Secretary,0.201653,0.202685,0.333333,0.737671
2,1,Le Thanh,0.240094,0.240964,0.185829,0.666887
4,8,Nguyen Tien,0.070059,0.070776,0.117293,0.258128
5,18,Đà Nang,0.000000,0.000000,0.000000,0.000000


In [ ]:
# final2.merge(df, on="JobID")
final_df = df.merge(final2, on='Unnamed: 0')
final_df

,Unnamed: 0,name_x,email,mobile_number,skills,degree,no_of_pages,file_path,All,pdf_to_base64,clean_all,name_y,KNN,TF-IDF,CV,Final
0,0,Le Thanh,attentionocr@gmail.com,096-997-3012,"['Product owner', 'Ai', 'Tensorflow', 'Chemist...",None,5,cv\1.0.pdf,le thanh longresearch and developmentphone 096...,JVBERi0xLjQKJaGzxdcKMSAwIG9iag0KPDwvVHlwZS9DYX...,le thanh longresearch developmentphone 0969973...,Le Thanh,0.288022,0.288530,0.214120,0.790672
1,1,Le Thanh,attentionocr@gmail.com,096-997-3012,"['Ai', 'Certification', 'Tensorflow', 'Chemist...",['MLOPS'],6,cv\2.0.pdf,le thanh longresearch and developmentbirth 260...,JVBERi0xLjQKJcLB2s/OCgoxIDAgb2JqIDw8CiAgL1R5cG...,le thanh longresearch developmentbirth 2609200...,Le Thanh,0.240094,0.240964,0.185829,0.666887
2,6,Deputy Secretary,hoquocthienanh@gmail.com,0905500293,"['Pandas', 'Ai', 'Forecasting', 'Seaborn', 'En...","[""Bachelor's degree Information Technology""]",1,cv\HoQuocThienAnh.pdf,ho quoc thien anhvietnam da nang city hoquoct...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/KQovQ3...,ho quoc thien anhvietnam da nang city hoquocth...,Deputy Secretary,0.201653,0.202685,0.333333,0.737671
3,8,Nguyen Tien,dungbk.nguyentien@g,0973621049,"['Video', 'Testing', 'C', 'Servers', 'Try', 'J...",['uf0b1'],2,cv\Nguyen-Tien-Dung.pdf,nguyen tien dung 0973621049 maledeveloper du...,JVBERi0xLjcKJYGBgYEKCjQgMCBvYmoKPDwKL1R5cGUgL1...,nguyen tien dung 0973621049 maledeveloper dung...,Nguyen Tien,0.070059,0.070776,0.117293,0.258128
4,11,| linkedin.com/in/haiph02,haiphung15012002@gmail.com,9486019,"['English', 'Queries', 'Scheduling', 'Saas', '...",['Bachelor of Science in Information Technolog...,1,cv\PDH_resume.pdf,phung duc hai84 948601960 haiphung15012002gma...,JVBERi0xLjUKJb/3ov4KMyAwIG9iago8PCAvTGluZWFyaX...,phung duc hai84 948601960 haiphung15012002gmai...,| linkedin.com/in/haiph02,0.333333,0.333333,0.331294,0.997961
5,18,Đà Nang,tranghuyen9696@gmail.com,222 5469,"['English', 'Website', 'Java', 'Analysis', 'Pr...",['Bachelor of Information Technology Engineeri...,1,cv\TrangNguyenHuyen_Resume_Intern.pdf,nguyn huyn trangvietnam nang da nang city tr...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/KQovQ3...,nguyn huyn trangvietnam nang da nang city tran...,Đà Nang,0.000000,0.000000,0.000000,0.000000


In [ ]:
df.columns

Index(['Unnamed: 0', 'name', 'email', 'mobile_number', 'skills', 'degree',
       'no_of_pages', 'file_path', 'All', 'pdf_to_base64', 'clean_all'],
      dtype='object')

In [ ]:
final_df.columns

Index(['Unnamed: 0', 'name_x', 'email', 'mobile_number', 'skills', 'degree',
       'no_of_pages', 'file_path', 'All', 'pdf_to_base64', 'clean_all',
       'name_y', 'KNN', 'TF-IDF', 'CV', 'Final'],
      dtype='object')

In [ ]:
final_df

,Unnamed: 0,name_x,email,mobile_number,skills,degree,no_of_pages,file_path,All,pdf_to_base64,clean_all,name_y,KNN,TF-IDF,CV,Final
0,0,Le Thanh,attentionocr@gmail.com,096-997-3012,"['Product owner', 'Ai', 'Tensorflow', 'Chemist...",None,5,cv\1.0.pdf,le thanh longresearch and developmentphone 096...,JVBERi0xLjQKJaGzxdcKMSAwIG9iag0KPDwvVHlwZS9DYX...,le thanh longresearch developmentphone 0969973...,Le Thanh,0.288022,0.288530,0.214120,0.790672
1,1,Le Thanh,attentionocr@gmail.com,096-997-3012,"['Ai', 'Certification', 'Tensorflow', 'Chemist...",['MLOPS'],6,cv\2.0.pdf,le thanh longresearch and developmentbirth 260...,JVBERi0xLjQKJcLB2s/OCgoxIDAgb2JqIDw8CiAgL1R5cG...,le thanh longresearch developmentbirth 2609200...,Le Thanh,0.240094,0.240964,0.185829,0.666887
2,6,Deputy Secretary,hoquocthienanh@gmail.com,0905500293,"['Pandas', 'Ai', 'Forecasting', 'Seaborn', 'En...","[""Bachelor's degree Information Technology""]",1,cv\HoQuocThienAnh.pdf,ho quoc thien anhvietnam da nang city hoquoct...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/KQovQ3...,ho quoc thien anhvietnam da nang city hoquocth...,Deputy Secretary,0.201653,0.202685,0.333333,0.737671
3,8,Nguyen Tien,dungbk.nguyentien@g,0973621049,"['Video', 'Testing', 'C', 'Servers', 'Try', 'J...",['uf0b1'],2,cv\Nguyen-Tien-Dung.pdf,nguyen tien dung 0973621049 maledeveloper du...,JVBERi0xLjcKJYGBgYEKCjQgMCBvYmoKPDwKL1R5cGUgL1...,nguyen tien dung 0973621049 maledeveloper dung...,Nguyen Tien,0.070059,0.070776,0.117293,0.258128
4,11,| linkedin.com/in/haiph02,haiphung15012002@gmail.com,9486019,"['English', 'Queries', 'Scheduling', 'Saas', '...",['Bachelor of Science in Information Technolog...,1,cv\PDH_resume.pdf,phung duc hai84 948601960 haiphung15012002gma...,JVBERi0xLjUKJb/3ov4KMyAwIG9iago8PCAvTGluZWFyaX...,phung duc hai84 948601960 haiphung15012002gmai...,| linkedin.com/in/haiph02,0.333333,0.333333,0.331294,0.997961
5,18,Đà Nang,tranghuyen9696@gmail.com,222 5469,"['English', 'Website', 'Java', 'Analysis', 'Pr...",['Bachelor of Information Technology Engineeri...,1,cv\TrangNguyenHuyen_Resume_Intern.pdf,nguyn huyn trangvietnam nang da nang city tr...,JVBERi0xLjQKMSAwIG9iago8PAovVGl0bGUgKP7/KQovQ3...,nguyn huyn trangvietnam nang da nang city tran...,Đà Nang,0.000000,0.000000,0.000000,0.000000
